In [5]:
# Cellule 1 : Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [6]:
# Cellule 2 : Chargement et préparation
print("Chargement des données...")
df = pd.read_csv('data_processed/allocine_cleaned.csv')

# Features de base
X = pd.DataFrame()

# Features principales
X['nb_actors'] = df['actors'].str.count(',') + 1
X['length'] = df['length'].fillna(df['length'].median())

# Périodes
df['date'] = pd.to_datetime(df['date'])
X['month'] = df['date'].dt.month
X['is_summer'] = df['date'].dt.month.isin([6,7,8]).astype(int)
X['is_holiday'] = df['date'].dt.month.isin([7,8,12]).astype(int)

# Genres
def has_genre(x, genre):
    try:
        genres = eval(x)
        return int(genre in genres)
    except:
        return 0

for genre in ['Drame', 'Action', 'Aventure', 'Comédie']:  # Ajout de Comédie
    X[f'has_{genre}'] = df['genre'].apply(lambda x: has_genre(x, genre))

# Features dérivées
X['actors_length_ratio'] = X['nb_actors'] / X['length']
X['drama_actors'] = X['has_Drame'] * X['nb_actors']
X['summer_adventure'] = X['is_summer'] * X['has_Aventure']
X['holiday_action'] = X['is_holiday'] * X['has_Action']

# Target avec transformation log
y = np.log1p(df['french_boxoffice'])

# Debug prints
print("\nStatistiques des entrées (log):")
print(y.describe())

print("\nDistribution des caractéristiques:")
print(f"Films en été: {X['is_summer'].sum()}")
print(f"Films en vacances: {X['is_holiday'].sum()}")

print("\nDistribution des genres:")
for genre in ['Drame', 'Action', 'Aventure', 'Comédie']:
    count = X[f'has_{genre}'].sum()
    print(f"{genre}: {count} films")

print("\nStatistiques des features numériques:")
print(X[['nb_actors', 'length', 'actors_length_ratio']].describe())

# Split des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nTaille des sets:")
print(f"Training: {X_train.shape}")
print(f"Test: {X_test.shape}")

Chargement des données...

Statistiques des entrées (log):
count    7292.000000
mean       11.924368
std         1.629329
min         8.778172
25%        10.680562
50%        11.919803
75%        13.124052
max        16.827512
Name: french_boxoffice, dtype: float64

Distribution des caractéristiques:
Films en été: 1752
Films en vacances: 1787

Distribution des genres:
Drame: 3187 films
Action: 841 films
Aventure: 698 films
Comédie: 2117 films

Statistiques des features numériques:
         nb_actors       length  actors_length_ratio
count  7292.000000  7292.000000          7292.000000
mean     13.674849   104.520433             0.128902
std       9.091951    18.952322             0.081633
min       1.000000    24.000000             0.005988
25%       6.000000    93.000000             0.060345
50%      12.000000   102.000000             0.119565
75%      20.000000   114.000000             0.181818
max      40.000000   272.000000             0.435294

Taille des sets:
Training: (5833, 13

In [7]:
# Cellule 3 : Modèle LightGBM
model = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    max_depth=8,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1
)

print("Entraînement du modèle...")
model.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='rmse',
)

Entraînement du modèle...


LGBMRegressor(colsample_bytree=0.8, learning_rate=0.05, max_depth=8,
              n_estimators=500, random_state=42, subsample=0.8, verbose=-1)

In [8]:
# Cellule 4 : Évaluation
y_pred = model.predict(X_test)

# Transformation inverse pour les métriques
y_test_orig = np.expm1(y_test)
y_pred_orig = np.expm1(y_pred)

# Métriques globales
r2 = r2_score(y_test_orig, y_pred_orig)
rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
mae = mean_absolute_error(y_test_orig, y_pred_orig)

print("\n=== PERFORMANCE GLOBALE DU MODÈLE ===")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.0f} entrées")
print(f"MAE: {mae:.0f} entrées")

print("\n=== IMPORTANCE DES FEATURES ===")
importances = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)
print(importances)

print("\n=== EXEMPLES D'UTILISATION ===")
# Exemple 1: Film d'action en été
example1 = pd.DataFrame({
    'nb_actors': [10],
    'length': [120],
    'month': [7],
    'is_summer': [1],
    'is_holiday': [1],
    'has_Drame': [0],
    'has_Action': [1],
    'has_Aventure': [0],
    'has_Comédie': [0],
    'actors_length_ratio': [10/120],
    'drama_actors': [0],
    'summer_adventure': [0],
    'holiday_action': [1]
})

# Exemple 2: Film d'aventure familial pour les vacances
example2 = pd.DataFrame({
    'nb_actors': [20],
    'length': [150],
    'month': [7],
    'is_summer': [1],
    'is_holiday': [1],
    'has_Drame': [0],
    'has_Action': [0],
    'has_Aventure': [1],
    'has_Comédie': [1],
    'actors_length_ratio': [20/150],
    'drama_actors': [0],
    'summer_adventure': [1],
    'holiday_action': [0]
})

print("1. Pour un film d'action standard (120min, 10 acteurs) en été:")
pred1 = np.expm1(model.predict(example1)[0])
print(f"Prédiction: {pred1:.0f} entrées")

print("\n2. Pour un film d'aventure familial (150min, 20 acteurs) en été:")
pred2 = np.expm1(model.predict(example2)[0])
print(f"Prédiction: {pred2:.0f} entrées")

# Analyse des erreurs
print("\n=== ANALYSE DES ERREURS ===")
errors = abs(y_test_orig - y_pred_orig)
print("\nDistribution des erreurs:")
print(pd.Series(errors).describe())

# Analyse par période
print("\nErreur moyenne par période:")
print(f"Été: {mean_absolute_error(y_test_orig[X_test['is_summer'] == 1], y_pred_orig[X_test['is_summer'] == 1]):.0f} entrées")
print(f"Vacances: {mean_absolute_error(y_test_orig[X_test['is_holiday'] == 1], y_pred_orig[X_test['is_holiday'] == 1]):.0f} entrées")

# Analyse par genre
print("\nErreur moyenne par genre:")
for genre in ['Drame', 'Action', 'Aventure', 'Comédie']:
    mask = X_test[f'has_{genre}'] == 1
    err = mean_absolute_error(y_test_orig[mask], y_pred_orig[mask])
    print(f"{genre}: {err:.0f} entrées")


=== PERFORMANCE GLOBALE DU MODÈLE ===
R² Score: 0.2342
RMSE: 946908 entrées
MAE: 384549 entrées

=== IMPORTANCE DES FEATURES ===
                feature  importance
9   actors_length_ratio        3500
1                length        3370
2                 month        1626
0             nb_actors        1461
10         drama_actors         802
8           has_Comédie         311
5             has_Drame         214
4            is_holiday         152
6            has_Action         151
7          has_Aventure         140
3             is_summer         120
11     summer_adventure          53
12       holiday_action          44

=== EXEMPLES D'UTILISATION ===
1. Pour un film d'action standard (120min, 10 acteurs) en été:
Prédiction: 232715 entrées

2. Pour un film d'aventure familial (150min, 20 acteurs) en été:
Prédiction: 966695 entrées

=== ANALYSE DES ERREURS ===

Distribution des erreurs:
count    1.459000e+03
mean     3.845492e+05
std      8.656041e+05
min      8.662686e+01
25%    